In [1]:
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import holidays
import os
import pickle
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns 
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import font_manager,rc
import platform
# 한글 폰트 설정
if platform.system() == 'Windows':# 윈도우인 경우
    font_path = "c:/Windows/Fonts/malgun.ttf" # 맑은 고딕 폰트 경로
    font_name = font_manager.FontProperties(fname=font_path).get_name()
    plt.rc('font', family=font_name)
# 음수 기호 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

In [4]:
data_path = "C:/Users/choin/Downloads/FitBit"

### Weight 데이터

In [6]:
weight = pd.read_csv(os.path.join(data_path, 'weightLogInfo_merged.csv'))
weight.head()

,Id,Date,WeightKg,WeightPounds,Fat,BMI,IsManualReport,LogId
0,1503960366,5/2/2016 11:59:59 PM,52.599998,115.963147,22.0,22.650000,True,1462233599000
1,1503960366,5/3/2016 11:59:59 PM,52.599998,115.963147,NaN,22.650000,True,1462319999000
2,1927972279,4/13/2016 1:08:52 AM,133.500000,294.317120,NaN,47.540001,False,1460509732000
3,2873212765,4/21/2016 11:59:59 PM,56.700001,125.002104,NaN,21.450001,True,1461283199000
4,2873212765,5/12/2016 11:59:59 PM,57.299999,126.324875,NaN,21.690001,True,1463097599000


In [8]:
weight['Date'] = pd.to_datetime(weight['Date']).dt.normalize()
weight = weight[['Id', 'Date', 'WeightKg']]
weight.head()

,Id,Date,WeightKg
0,1503960366,2016-05-02,52.599998
1,1503960366,2016-05-03,52.599998
2,1927972279,2016-04-13,133.500000
3,2873212765,2016-04-21,56.700001
4,2873212765,2016-05-12,57.299999


In [11]:
weight['Id'].value_counts()

Id
6962181067    30
8877689391    24
4558609924     5
1503960366     2
2873212765     2
4319703577     2
1927972279     1
5577150313     1
Name: count, dtype: int64

In [32]:
weight = pd.read_csv('weightLogInfo_merged.csv')
weight.head()

,Id,Date,WeightKg,WeightPounds,Fat,BMI,IsManualReport,LogId
0,1503960366,5/2/2016 11:59:59 PM,52.599998,115.963147,22.0,22.650000,True,1462233599000
1,1503960366,5/3/2016 11:59:59 PM,52.599998,115.963147,NaN,22.650000,True,1462319999000
2,1927972279,4/13/2016 1:08:52 AM,133.500000,294.317120,NaN,47.540001,False,1460509732000
3,2873212765,4/21/2016 11:59:59 PM,56.700001,125.002104,NaN,21.450001,True,1461283199000
4,2873212765,5/12/2016 11:59:59 PM,57.299999,126.324875,NaN,21.690001,True,1463097599000


In [33]:
weight['Id'].value_counts()

Id
6962181067    30
8877689391    24
4558609924     5
1503960366     2
2873212765     2
4319703577     2
1927972279     1
5577150313     1
Name: count, dtype: int64

### Sleep 데이터

In [9]:
sleep = pd.read_csv(os.path.join(data_path, 'sleepDay_merged.csv'))
sleep.head()

,Id,SleepDay,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed
0,1503960366,4/12/2016 12:00:00 AM,1,327,346
1,1503960366,4/13/2016 12:00:00 AM,2,384,407
2,1503960366,4/15/2016 12:00:00 AM,1,412,442
3,1503960366,4/16/2016 12:00:00 AM,2,340,367
4,1503960366,4/17/2016 12:00:00 AM,1,700,712


In [10]:
sleep['Date'] = pd.to_datetime(sleep['SleepDay']).dt.normalize()
sleep = sleep[['Id', 'Date', 'TotalMinutesAsleep', 'TotalTimeInBed']]
sleep.head()

C:\Users\choin\AppData\Local\Temp\ipykernel_8144\2520832700.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  sleep['Date'] = pd.to_datetime(sleep['SleepDay']).dt.normalize()


,Id,Date,TotalMinutesAsleep,TotalTimeInBed
0,1503960366,2016-04-12,327,346
1,1503960366,2016-04-13,384,407
2,1503960366,2016-04-15,412,442
3,1503960366,2016-04-16,340,367
4,1503960366,2016-04-17,700,712


In [14]:
sleep['Id'].value_counts()

Id
8378563200    32
6962181067    31
5553957443    31
4702921684    28
2026352035    28
3977333714    28
4445114986    28
5577150313    26
4319703577    26
1503960366    25
7086361926    24
4388161847    24
6117666160    18
8792009665    15
2347167796    15
4020332650     8
1927972279     5
4558609924     5
1644430081     4
6775888955     3
8053475328     3
1844505072     3
7007744171     2
2320127002     1
Name: count, dtype: int64

### Activity 데이터

In [15]:
activity = pd.read_csv(os.path.join(data_path, 'dailyActivity_merged.csv'))
activity.head()

,Id,ActivityDate,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories
0,1503960366,4/12/2016,13162,8.50,8.50,0.0,1.88,0.55,6.06,0.0,25,13,328,728,1985
1,1503960366,4/13/2016,10735,6.97,6.97,0.0,1.57,0.69,4.71,0.0,21,19,217,776,1797
2,1503960366,4/14/2016,10460,6.74,6.74,0.0,2.44,0.40,3.91,0.0,30,11,181,1218,1776
3,1503960366,4/15/2016,9762,6.28,6.28,0.0,2.14,1.26,2.83,0.0,29,34,209,726,1745
4,1503960366,4/16/2016,12669,8.16,8.16,0.0,2.71,0.41,5.04,0.0,36,10,221,773,1863


In [16]:
activity['Date'] = pd.to_datetime(activity['ActivityDate']).dt.normalize()
activity = activity[['Id', 'Date', 'TotalSteps', 'VeryActiveMinutes', 'FairlyActiveMinutes', 'LightlyActiveMinutes',
                           'SedentaryMinutes', 'VeryActiveDistance', 'ModeratelyActiveDistance', 'LightActiveDistance', 'Calories']]
activity.head()

,Id,Date,TotalSteps,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,Calories
0,1503960366,2016-04-12,13162,25,13,328,728,1.88,0.55,6.06,1985
1,1503960366,2016-04-13,10735,21,19,217,776,1.57,0.69,4.71,1797
2,1503960366,2016-04-14,10460,30,11,181,1218,2.44,0.40,3.91,1776
3,1503960366,2016-04-15,9762,29,34,209,726,2.14,1.26,2.83,1745
4,1503960366,2016-04-16,12669,36,10,221,773,2.71,0.41,5.04,1863


In [21]:
activity['Id'].value_counts()

Id
1503960366    31
4319703577    31
8583815059    31
8378563200    31
8053475328    31
7086361926    31
6962181067    31
5553957443    31
4702921684    31
4558609924    31
1624580081    31
4388161847    31
4445114986    31
8877689391    31
1927972279    31
2873212765    31
2320127002    31
4020332650    31
2026352035    31
1844505072    31
2022484408    31
3977333714    30
1644430081    30
5577150313    30
8792009665    29
6290855005    29
6117666160    28
6775888955    26
7007744171    26
3372868164    20
8253242879    19
2347167796    18
4057192912     4
Name: count, dtype: int64

### 통합

In [41]:
df_merged = activity.merge(sleep, on=['Id', 'Date'], how='left')
df_merged

,Id,Date,TotalSteps,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,Calories,TotalMinutesAsleep,TotalTimeInBed
0,1503960366,2016-04-12,13162,25,13,328,728,1.88,0.55,6.06,1985,327.0,346.0
1,1503960366,2016-04-13,10735,21,19,217,776,1.57,0.69,4.71,1797,384.0,407.0
2,1503960366,2016-04-14,10460,30,11,181,1218,2.44,0.40,3.91,1776,NaN,NaN
3,1503960366,2016-04-15,9762,29,34,209,726,2.14,1.26,2.83,1745,412.0,442.0
4,1503960366,2016-04-16,12669,36,10,221,773,2.71,0.41,5.04,1863,340.0,367.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,8877689391,2016-05-08,10686,17,4,245,1174,1.08,0.20,6.80,2847,NaN,NaN
939,8877689391,2016-05-09,20226,73,19,217,1131,11.10,0.80,6.24,3710,NaN,NaN
940,8877689391,2016-05-10,10733,18,11,224,1187,1.35,0.46,6.28,2832,NaN,NaN
941,8877689391,2016-05-11,21420,88,12,213,1127,13.22,0.41,5.89,3832,NaN,NaN


In [42]:
df_merged['Id'].value_counts()

Id
8378563200    32
4702921684    32
4388161847    32
1503960366    31
4319703577    31
8583815059    31
8053475328    31
7086361926    31
6962181067    31
5553957443    31
4558609924    31
1624580081    31
4445114986    31
8877689391    31
1927972279    31
2873212765    31
2320127002    31
4020332650    31
2026352035    31
1844505072    31
2022484408    31
3977333714    30
1644430081    30
5577150313    30
8792009665    29
6290855005    29
6117666160    28
6775888955    26
7007744171    26
3372868164    20
8253242879    19
2347167796    18
4057192912     4
Name: count, dtype: int64

In [43]:
df_merged.isnull().sum()

Id                            0
Date                          0
TotalSteps                    0
VeryActiveMinutes             0
FairlyActiveMinutes           0
LightlyActiveMinutes          0
SedentaryMinutes              0
VeryActiveDistance            0
ModeratelyActiveDistance      0
LightActiveDistance           0
Calories                      0
TotalMinutesAsleep          530
TotalTimeInBed              530
dtype: int64

In [45]:
# 보간 + 필요시 전방 채움
df_merged['TotalMinutesAsleep'] = (
    df_merged.groupby('Id')['TotalMinutesAsleep']
    .transform(lambda x: x.interpolate())
    .ffill()
    .bfill()
)
df_merged[['TotalMinutesAsleep','TotalTimeInBed']].isna().sum()

TotalMinutesAsleep    0
TotalTimeInBed        0
dtype: int64

In [46]:
df_merged

,Id,Date,TotalSteps,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,Calories,TotalMinutesAsleep,TotalTimeInBed
0,1503960366,2016-04-12,13162,25,13,328,728,1.88,0.55,6.06,1985,327.0,346.0
1,1503960366,2016-04-13,10735,21,19,217,776,1.57,0.69,4.71,1797,384.0,407.0
2,1503960366,2016-04-14,10460,30,11,181,1218,2.44,0.40,3.91,1776,398.0,424.5
3,1503960366,2016-04-15,9762,29,34,209,726,2.14,1.26,2.83,1745,412.0,442.0
4,1503960366,2016-04-16,12669,36,10,221,773,2.71,0.41,5.04,1863,340.0,367.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,8877689391,2016-05-08,10686,17,4,245,1174,1.08,0.20,6.80,2847,439.0,463.0
939,8877689391,2016-05-09,20226,73,19,217,1131,11.10,0.80,6.24,3710,439.0,463.0
940,8877689391,2016-05-10,10733,18,11,224,1187,1.35,0.46,6.28,2832,439.0,463.0
941,8877689391,2016-05-11,21420,88,12,213,1127,13.22,0.41,5.89,3832,439.0,463.0


In [48]:
file_name = "merged_fitbit.csv"
df_merged.to_csv(f"{data_path}/{file_name}", index=False)

In [ ]:
with open("fitbit_merged.pkl", "wb") as f:
    pickle.dump(df_merged, f)

In [50]:
# Pickle 파일 저장
file_name = "fitbit_merged.pkl"
file_path = os.path.join(data_path, file_name)

# 경로 없으면 생성
#os.makedirs(data_path, exist_ok=True)

# Pickle로 저장
with open(file_path, "wb") as f:
    pickle.dump(df_merged, f)

In [ ]:
# 저장된 파일 불러오는 방법
with open("C:/Users/choin/Downloads/FitBit/fitbit_merged.pkl", "rb") as f:
    df_merged = pickle.load(f)